# Filtering rows 3: multiple filter conditions
By the end of this lecture you will be able to:
- use multiple AND conditions in `filter`
- use multiple OR conditions in `filter`
- optimise multiple conditions in lazy mode

In [1]:
import polars as pl

In [2]:
csv_file = "data_titanic.csv"

In [4]:
df = pl.scan_csv(csv_file)

## Multiple conditions

### Apply `AND` conditions

We can apply filter `AND` conditions where all conditions must be met in a number of ways.

The first way is to **chaining** multiple calls to `filter`.

In this example we keep all first class passengers that are over 70

In [5]:
(
    df
    .filter(
        Pclass = 1
    )
    .filter(
        pl.col('Age') > 70
    )
    .head(3)
)

<LazyFrame [12 cols, {"PassengerId": Int64 … "Embarked": String}] at 0x279DD52F610>

In eager mode chaining is inefficient. For each call to `filter` Polars has to do a full pass through the rows of the `DataFrame`. It is better to combine everything into a single condition 

One way to do this is to **concatenate** multiple `AND` conditions in a single `filter` call using `&`

In [6]:
(
    df
    .filter(
        (pl.col('Age') > 70) & (pl.col('Pclass') == 1)
    )
    .head(2)
)

<LazyFrame [12 cols, {"PassengerId": Int64 … "Embarked": String}] at 0x279DD52E560>

There is a less verbose way to do this by passing the predicates as a comma-separated list of expressions

In [12]:
(
    df
    .filter(
        pl.col("Pclass") == 1,
        pl.col("Age") > 30
    )
    .head(10)
).collect()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
7,0,1,"""McCarthy, Mr. …","""male""",54.0,0,0,"""17463""",51.8625,"""E46""","""S"""
12,1,1,"""Bonnell, Miss.…","""female""",58.0,0,0,"""113783""",26.55,"""C103""","""S"""
31,0,1,"""Uruchurtu, Don…","""male""",40.0,0,0,"""PC 17601""",27.7208,null,"""C"""
36,0,1,"""Holverson, Mr.…","""male""",42.0,1,0,"""113789""",52.0,null,"""S"""
53,1,1,"""Harper, Mrs. H…","""female""",49.0,1,0,"""PC 17572""",76.7292,"""D33""","""C"""
55,0,1,"""Ostby, Mr. Eng…","""male""",65.0,0,1,"""113509""",61.9792,"""B30""","""C"""
62,1,1,"""Icard, Miss. A…","""female""",38.0,0,0,"""113572""",80.0,"""B28""",null


If we are applying multiple *equality* conditions we can do this with keywords (note the single `=` in this format)

In [13]:
(
    df
    .filter(
        Pclass = 1,
        Age = 70
    )
)

<LazyFrame [12 cols, {"PassengerId": Int64 … "Embarked": String}] at 0x279DD4D8820>

### Apply an AND condition using `pl.all_horizontal`
Specifying multiple conditions in chained `filters` or using `&` is fine when we have a small number of conditions to apply. However, we can use the `pl.all_horizontal` method when we want to apply an AND condition on many columns.

> The methods above with a comma-separate list of conditions are equivalent to `pl.all_horizontal`

In this example we:
- first call `pl.all().is_not_null()` to create a Boolean `DataFrame` where each call is True if the underlying value is not `null`
- then call `pl.all_horizontal` to find rows where all values are `True` (i.e. all values are not `null`)

In [17]:
(
    df
    .with_row_index('index')
    .filter(
        pl.all_horizontal(
            pl.all().is_not_null()
        )
    )
    .head(10)
).collect()

index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
6,7,0,1,"""McCarthy, Mr. …","""male""",54.0,0,0,"""17463""",51.8625,"""E46""","""S"""
10,11,1,3,"""Sandstrom, Mis…","""female""",4.0,1,1,"""PP 9549""",16.7,"""G6""","""S"""
11,12,1,1,"""Bonnell, Miss.…","""female""",58.0,0,0,"""113783""",26.55,"""C103""","""S"""
21,22,1,2,"""Beesley, Mr. L…","""male""",34.0,0,0,"""248698""",13.0,"""D56""","""S"""
23,24,1,1,"""Sloper, Mr. Wi…","""male""",28.0,0,0,"""113788""",35.5,"""A6""","""S"""
27,28,0,1,"""Fortune, Mr. C…","""male""",19.0,3,2,"""19950""",263.0,"""C23 C25 C27""","""S"""
52,53,1,1,"""Harper, Mrs. H…","""female""",49.0,1,0,"""PC 17572""",76.7292,"""D33""","""C"""


### Apply `AND` condition on a range

We use `in_between` to apply a condition on a range. In this case we are looking for values **greater than or equal to** 10 and **less than or equal to** 13

In [18]:
(
    df
    .filter(
        pl.col("Age").is_between(10,13)
    )
    .head(2)
)

<LazyFrame [12 cols, {"PassengerId": Int64 … "Embarked": String}] at 0x279DD89FE20>

We use the `closed` argument to specify if we want the range to be open, closed on both sides or open on the left or right. The default is for the range to be closed (with a value of `"both"`). 

In this example we are looking for values from 10 to 13 exclusive of the boundaries

In [19]:
(
    df
    .filter(
        pl.col("Age").is_between(10,13,closed="none")
    )
    .head(2)
)

<LazyFrame [12 cols, {"PassengerId": Int64 … "Embarked": String}] at 0x279DD8A0970>

### Apply `OR` conditions

We can apply an OR filter using the pipe `|` operator.

In this example we look for rows where the passenger is over 70 OR the passenger is in first class

In [20]:
(
    df
    .filter(
        (pl.col('Age') > 70) | (pl.col('Pclass') == 1)
    )
    .head(2)
)

<LazyFrame [12 cols, {"PassengerId": Int64 … "Embarked": String}] at 0x279DD89FBB0>

One kind of OR condition is when we want to check if a row is equal to any value in a `list`. We can do this with `is_in`

In [21]:
(
    df
    .filter(
        pl.col('Pclass').is_in([2,3])
    )
    .head(3)
)

<LazyFrame [12 cols, {"PassengerId": Int64 … "Embarked": String}] at 0x279DBE40E80>

### Multiple conditions in lazy mode
In *lazy mode* if we pass multiple `filter` calls then the query optimizer combines these into a *single condition* inside `SELECTION`.

In this example we filter for first class passengers over the age of 70.

In [22]:
df = (
    pl.scan_csv(csv_file)
    .filter(
        pl.col('Pclass')==1
    )
    .filter(
        (pl.col('Age') > 70)
    )
)
print(df.explain())


  Csv SCAN data_titanic.csv
  PROJECT */12 COLUMNS
  SELECTION: [([(col("Age")) > (70.0)]) & ([(col("Pclass")) == (1)])]


In the query plan we see the conditionsa are combined to a single condition by the query optimiser

## Exercises
In the exercises you will develop your understanding of:
- applying multiple AND conditions
- applying multiple OR conditions

### Exercise 1 
Filter the `DataFrame` to find rows where `Age` is between 30 and 50 (including the lower bound) and the passenger is in 2nd class. Do this in eager mode in a single pass through the `DataFrame`

In [27]:
(
    pl.read_csv(csv_file)
    .filter(pl.col('Age').is_between(30,50), pl.col("Pclass") == 2)
    .head(10)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
21,0,2,"""Fynney, Mr. Jo…","""male""",35.0,0,0,"""239865""",26.0,null,"""S"""
22,1,2,"""Beesley, Mr. L…","""male""",34.0,0,0,"""248698""",13.0,"""D56""","""S"""
71,0,2,"""Jenkin, Mr. St…","""male""",32.0,0,0,"""C.A. 33111""",10.5,null,"""S"""
99,1,2,"""Doling, Mrs. J…","""female""",34.0,0,1,"""231919""",23.0,null,"""S"""
100,0,2,"""Kantor, Mr. Si…","""male""",34.0,1,0,"""244367""",26.0,null,"""S"""
123,0,2,"""Nasser, Mr. Ni…","""male""",32.5,1,0,"""237736""",30.0708,null,"""C"""
124,1,2,"""Webber, Miss. …","""female""",32.5,0,0,"""27267""",13.0,"""E101""","""S"""
149,0,2,"""Navratil, Mr. …","""male""",36.5,0,2,"""230080""",26.0,"""F2""","""S"""
150,0,2,"""Byles, Rev. Th…","""male""",42.0,0,0,"""244310""",13.0,null,"""S"""


Do this again combining the range condition with the keyword approach for the 2nd class condition - does the order you pass the conditions matter?

In [28]:
(
    pl.read_csv(csv_file)
    .filter(pl.col('Age').is_between(30, 50), Pclass = 2)
    .head(10)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
21,0,2,"""Fynney, Mr. Jo…","""male""",35.0,0,0,"""239865""",26.0,null,"""S"""
22,1,2,"""Beesley, Mr. L…","""male""",34.0,0,0,"""248698""",13.0,"""D56""","""S"""
71,0,2,"""Jenkin, Mr. St…","""male""",32.0,0,0,"""C.A. 33111""",10.5,null,"""S"""
99,1,2,"""Doling, Mrs. J…","""female""",34.0,0,1,"""231919""",23.0,null,"""S"""
100,0,2,"""Kantor, Mr. Si…","""male""",34.0,1,0,"""244367""",26.0,null,"""S"""
123,0,2,"""Nasser, Mr. Ni…","""male""",32.5,1,0,"""237736""",30.0708,null,"""C"""
124,1,2,"""Webber, Miss. …","""female""",32.5,0,0,"""27267""",13.0,"""E101""","""S"""
149,0,2,"""Navratil, Mr. …","""male""",36.5,0,2,"""230080""",26.0,"""F2""","""S"""
150,0,2,"""Byles, Rev. Th…","""male""",42.0,0,0,"""244310""",13.0,null,"""S"""


### Exercise 2
Return all the rows of the `DataFrame` where at least one column on the row is `null` (excluding the `Cabin` column with many `null` values)

In [ ]:
(
    pl.read_csv(csv_file)
    .drop("Cabin")
    <blank>
)

### Exercise 3
Create a `DataFrame` where the passengers got on in Cork ("C") or Southampton ("S") using the pipe operator

In [ ]:
(
    pl.read_csv(csv_file)
    <blank>
)

Do this again using the `is_in` approach

### Exercise 4
Load the Spotify CSV data into a `DataFrame`

In [ ]:
pl.Config.set_fmt_str_lengths(30)
spotify_csv = "../data/spotify-charts-2017-2021-global-top200.csv.gz"
spotify_df = pl.read_csv(spotify_csv)
spotify_df.head()

Find all rows where the number of streams is greater than 10 million and the trend is "NEW_ENTRY"  

Find the rows where the artist is either Drake or Ed Sheeran and the rank is less than (better than) 5

## Solutions
### Solution to Exercise 1
Create a `DataFrame` where `Age` is between 30 and 50 (including the lower bound) and the passenger is in 2nd class. Do this in eager mode in a single pass through the `DataFrame`

In [29]:
(
    pl.read_csv(csv_file)
    .filter(
        (pl.col('Age').is_between(30,50,closed="left")) & (pl.col('Pclass')==2)
    )
    .head()
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
21,0,2,"""Fynney, Mr. Jo…","""male""",35.0,0,0,"""239865""",26.0,null,"""S"""
22,1,2,"""Beesley, Mr. L…","""male""",34.0,0,0,"""248698""",13.0,"""D56""","""S"""
71,0,2,"""Jenkin, Mr. St…","""male""",32.0,0,0,"""C.A. 33111""",10.5,null,"""S"""
99,1,2,"""Doling, Mrs. J…","""female""",34.0,0,1,"""231919""",23.0,null,"""S"""
100,0,2,"""Kantor, Mr. Si…","""male""",34.0,1,0,"""244367""",26.0,null,"""S"""


Do this again combining the range condition with the keyword approach for the 2nd class condition - does the order you pass the conditions matter?

In [30]:
(
    pl.read_csv(csv_file)
    .filter(
        pl.col('Age').is_between(30,50,closed="left"),
        Pclass=2,
    )
    .head()
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
21,0,2,"""Fynney, Mr. Jo…","""male""",35.0,0,0,"""239865""",26.0,null,"""S"""
22,1,2,"""Beesley, Mr. L…","""male""",34.0,0,0,"""248698""",13.0,"""D56""","""S"""
71,0,2,"""Jenkin, Mr. St…","""male""",32.0,0,0,"""C.A. 33111""",10.5,null,"""S"""
99,1,2,"""Doling, Mrs. J…","""female""",34.0,0,1,"""231919""",23.0,null,"""S"""
100,0,2,"""Kantor, Mr. Si…","""male""",34.0,1,0,"""244367""",26.0,null,"""S"""


The order you do this matters - Python keyword arguments must be the last on the list

### Solution to Exercise 2
Return all the rows of the `DataFrame` where at least one column on the row is `null` (excluding the `Cabin` column with many `null` values)

In [31]:
(
    pl.read_csv(csv_file)
    .drop("Cabin")
    .filter(
        pl.any_horizontal(
            pl.all().is_null()
        )
    )
    .head()
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str
6,0,3,"""Moran, Mr. Jam…","""male""",null,0,0,"""330877""",8.4583,"""Q"""
18,1,2,"""Williams, Mr. …","""male""",null,0,0,"""244373""",13.0,"""S"""
20,1,3,"""Masselmani, Mr…","""female""",null,0,0,"""2649""",7.225,"""C"""
27,0,3,"""Emir, Mr. Farr…","""male""",null,0,0,"""2631""",7.225,"""C"""
29,1,3,"""O'Dwyer, Miss.…","""female""",null,0,0,"""330959""",7.8792,"""Q"""


### Solution to Exercise 3
Create a `DataFrame` where the passengers got on in Cork ("C") or Southampton ("S") using the pipe operator

In [32]:
(
    pl.read_csv(csv_file)
    .filter(
        (pl.col('Embarked') == "C") | (pl.col('Embarked') == "S")
    )
    .head(3)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""



Do this again using the `is_in` approach

In [33]:
(
    pl.read_csv(csv_file)
    .filter(
        pl.col('Embarked').is_in(["C","S"])
    )
    .head()
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""
4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
5,0,3,"""Allen, Mr. Wil…","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


### Solution to Exercise 4
Load the Spotify CSV data into a `DataFrame`

In [ ]:
pl.Config.set_fmt_str_lengths(30)
spotify_csv = "../data/spotify-charts-2017-2021-global-top200.csv.gz"
spotify_df = pl.read_csv(spotify_csv)
spotify_df.head()

Find all rows where the number of streams is greater than 10 million and the trend is "NEW_ENTRY"  

In [ ]:
(
    spotify_df
    .filter(
        pl.col("streams")>1e7,
        trend = "NEW_ENTRY"
    )
)

Find the rows where the artist is either Drake or Ed Sheeran and the rank is less than (better than) 5

In [ ]:
(
    spotify_df
    .filter(
        pl.col("artist").is_in(["Drake","Ed Sheeran"])
    )
    .filter(pl.col("rank")<5)
)